In [114]:
import pandas as pd
import numpy as np
from linreg import linreg_post
from norm1d import *

# each vote by each legislator gets a z score, the utility from the vote
model_df = pd.DataFrame.from_csv("model_position_df.df").head(1000)
model_df["z"] = np.random.uniform(-1,1, model_df.shape[0])

# legislators
leg = pd.DataFrame(model_df["legislator"].unique(), columns=["legislator"])
leg["theta"] = np.random.uniform(-1,1, leg.shape[0])

# votes
votes = pd.DataFrame(model_df["vote"].unique(), columns=["vote"])
votes["alpha"] =  np.random.uniform(-1,1, votes.shape[0])
votes["beta"] = np.random.uniform(-1,1, votes.shape[0])

In [133]:
def draw_z(row):
    """
    """
    voteinfo = votes.query("vote=={}".format(row["vote"]))
    theta = leg.query("legislator=={}".format(row["legislator"]))["theta"]
    theta_float = theta[theta.keys()[0]]   # this is a float! 
    leginfo = leg.query("legislator=={}".format(row["legislator"]))
    mean =  voteinfo["alpha"] + voteinfo["beta"] * theta_float
    mean_float = mean[mean.keys()[0]]
    standard_deviation = 1
    
    if row["position"] == 1:
        return truncnormal(mean_float, standard_deviation, 0, 100)
    elif row["position"] == 0:
        return truncnormal(mean_float, standard_deviation, -100, 0)
    assert "do not" == "get to here"

def draw_theta(row):
    """
    Draw a theta
    """
    ls_votes = model_df.query("legislator=={}".format(row["legislator"]))
    ls_votes = ls_votes.merge(votes, on='vote')
    X = ls_votes[["alpha", "beta"]].as_matrix()
    Y = ls_votes["z"].as_matrix()
    m, cv = linreg_post(X, Y, np.zeros(2), 1, 1)
    return np.random.multivariate_normal(m, cv)[1]


def draw_ab(row):
    """
    Draw a theta
    """
    vote_i = model_df.query("vote=={}".format(row["vote"]))
    vote_i = vote_i.merge(leg, on="legislator")
    vote_i["dummy"] = 1
    X = vote_i[["dummy", "theta"]].as_matrix()
    Y = vote_i["z"]
    m, cv = linreg_post(X, Y, np.zeros(2), 1, 1)
    a, b = np.random.multivariate_normal(m, cv)
    vote = row["vote"]
    return pd.Series([vote, a, b], index=['vote', 'alpha_new', 'beta_new'])


def utility(row):
    if row["vote"] == 0:
        return np.log(normcdf(1 - row["z"]))
    elif row["vote"] == 1:
        return np.log(normcdf(row["z"]))

for i in range(10):
    # print i
    print i, np.sum(model_df.apply(utility, axis=1))

    #alpha, beta
    #new = votes.apply(draw_ab, axis=1)
    #votes = votes.merge(new, on="vote")[["vote", "alpha_new", "beta_new"]]
    #votes.columns = ["vote", "alpha", "beta"]

    #theta
    leg["theta"] = leg.apply(draw_theta, axis=1)

    #z
    model_df["z"] = model_df.apply(draw_z, axis=1)



0 -1239.02394881


/Library/Python/2.7/site-packages/ipykernel/__main__.py:27: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1 -1134.92313208
2 -1152.68221065
3 -1247.89180694
4 -1232.84138347
5 -1204.54239571
6 -1206.50595795
7 -1284.89149648
8 -1295.13863563
9 -1226.39845978
